<a href="https://colab.research.google.com/github/Annie00000/Project/blob/main/3_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1.

* is_test_running() 用于检查 test.py 进程是否在运行。它运行了一个 ps -ef 命令来获取系统中所有进程的列表，然后检查输出中是否包含了 test.py。如果包含，则返回 True，表示 test.py 进程正在运行；否则返回 False。

* subprocess.Popen : 这行代码创建了一个新的进程来运行 test.py 程序。subprocess.Popen() 函数用于启动一个新的进程。传递给它的参数是一个列表，第一个元素是要执行的程序名称，后面的元素是传递给程序的命令行参数。

In [ ]:
import time
import subprocess

# 检查 test.py 是否在运行的函数
def is_test_running():
    # 运行 ps 命令检查 test.py 是否在进程列表中
    ps_output = subprocess.run(['ps', '-ef'], capture_output=True, text=True)
    return 'test.py' in ps_output.stdout

if __name__ == "__main__":
    while True:
        # 检查 test.py 是否在运行
        if not is_test_running():
            # 启动 test.py
            subprocess.Popen(['python3', 'test.py'])
        time.sleep(600)  # 每隔10分钟检查一次


## 2.

'test.py' in ps_output.stdout:

 這句程式碼是在檢查一個字串 'test.py' 是否在字串 ps_output.stdout 中出現。 在這裡，ps_output.stdout 應該是一個字串，可能包含了系統的進程信息，透過運行 ps -ef 命令來獲取。 然後，程式會檢查是否有包含 'test.py' 的內容，以確定 test.py 進程是否正在執行中。

如果 'test.py' 出現在 ps_output.stdout 中，is_test_running() 函數傳回 True，表示 test.py 進程正在執行。 否則回傳 False，表示 test.py 進程沒有在運作。

這種方式通常用於在 Python 中檢查其他進程的運行狀態。

In [ ]:
import time
import subprocess
import multiprocessing as mp

# 定义 collection 函数
def collection(mode, i, args):
    if mode == 'today':
        print(f'Today mode: {i}')
        # 执行针对今天的任务逻辑
    elif mode == 'recent':
        print(f'Recent mode: {i}')
        # 执行针对最近的任务逻辑
    elif mode == 'past':
        print(f'Past mode: {i}')
        # 执行针对过去的任务逻辑

# 监控 test.py 进程是否在运行的函数
def is_test_running():
    # 运行 ps 命令检查 test.py 是否在进程列表中
    ps_output = subprocess.run(['ps', '-ef'], capture_output=True, text=True)
    return 'test.py' in ps_output.stdout
    """""""# 检查一个字符串 'test.py' 是否在字符串 ps_output.stdout 中出现

# 监控程序的逻辑
def monitor():
    while True:
        # 检查 test.py 是否在运行
        if not is_test_running():
            # 启动 test.py
            pool = mp.Pool(processes=3)
            for i, mode in enumerate(['today', 'recent', 'past']):
                # 使用 apply_async 方法异步提交任务
                pool.apply_async(collection, args=(mode, i, args))
            # 关闭进程池，阻止提交新任务
            pool.close()
            # 等待所有进程完成
            pool.join()
        time.sleep(600)  # 每隔10分钟检查一次

if __name__ == "__main__":
    monitor()  # 启动监控程序


## NOW

整体逻辑是当程序启动时，首先尝试获取锁，如果获取成功则继续执行程序；如果锁文件存在并且未超时，则程序会等待一段时间再次尝试获取锁，直到获取成功为止。获取锁成功后，程序进入主循环，执行 collection 函数。在 collection 函数中，不断更新锁文件的访问时间和修改时间，并根据不同的模式设置不同的睡眠时间。最后，无论程序执行成功还是出现异常，都会释放锁文件。

In [ ]:
from multiprocessing import Pool
import os
import time

# 定义锁文件路径和服务器标识文件路径
LOCK_FILE = "lock.txt"
LOCK_TIMEOUT = 600  # 设置锁超时时间（秒）

def acquire_lock(server_id):
    """获取锁，并记录服务器标识"""
    while True:
        if os.path.exists(LOCK_FILE):  # 检查是否存在锁文件
            if check_lock_timeout():  # 检查锁是否超时
                print("Lock timeout, trying to remove lock file...")
                release_lock()  # 释放过期的锁
                continue  # 继续循环，重新尝试获取锁 (跳回到 while True: 行，重新开始下一轮的循环。)

            print(f"Another server (ID: {server_id}) is running test.py. Waiting for 5 seconds...")
            time.sleep(5) #执行完毕后，程序会继续执行if os.path.exists(LOCK_FILE):
        else:
            try:
                with open(LOCK_FILE, "w") as f:
                    f.write(str(server_id))  # 写入当前服务器标识到锁文件
                break  # 获取锁成功，退出循环
            except Exception as e:
                print("Failed to acquire lock:", str(e))
                # 在这里处理获取锁失败的情况

def check_lock_timeout():
    """检查锁是否超时"""
    if os.path.exists(LOCK_FILE):
        lock_time = os.path.getmtime(LOCK_FILE)
        current_time = time.time()
        if current_time - lock_time > LOCK_TIMEOUT:
            return True
    return False

def release_lock():
    """释放锁"""
    try:
        if os.path.exists(LOCK_FILE):
            os.remove(LOCK_FILE)
    except Exception as e:
        print("Failed to release lock:", str(e))
        # 在这里处理释放锁失败的情况

def collection(mode):
  while True:
    os.utime(LOCK_FILE, (time.time(), time.time()))
    """执行 collection 函数，并在执行前后处理锁"""
    try:
        if mode=='today':
          sleep_time = 60
        elif mode=='recent':
          sleep_time = 60*10
        else:
          sleep_time = 60*60*3

        # 在这里执行 collection 函数的内容
        print(f"Process {option} (Server {server_id}) is running collection...")
        time.sleep(10)  # 模拟 collection 函数的执行时间

        time.sleep(sleep_time)

    except Exception as e:
        print("Error in collection:", str(e))
        # 在这里处理 collection 函数执行过程中的异常


if __name__ == "__main__":
    # 假设每个服务器有一个唯一的ID，可以根据需要自行获取
    server_id = "Server1"  # 你需要替换为服务器的实际标识

    acquire_lock()
    try:
      pool = Pool(processes=3)
      for i, mode in enumerate(['today', 'recent', 'past']):
          pool.apply_async(collection, args=(mode,))
      pool.close()
      pool.join()
    finally:
      release_lock()
